# CRUD
#### (Create,Read,Update,Delete)

### Inhold
1. Start
2. Hent Fra Database
    - Simple Queries
    - Data tranformation
    - Eager loading
3. Oprettelse
    - Insert af en entry.
    - Nested Insert.
    - Insert af flere entries.
4. Opdatering af data
    - Opdatering af en entry.
    - Opdatering med brug af ChangeTracker.
    - Opdatering af en detached entry.
----

## 1. Start
for at bruge denne notebook skal du have powershell core installeret.

Winget:

In [ ]:
# Out-file delen kan undværes hvis du copy paster den til din kommandopromt. da den spytter en masse ud i notedsbogen.
winget install Microsoft.PowerShell 
| Out-File -FilePath WingetOutput.txt

Kør koden nedenunder før du kører eksemplerne.

In [1]:

#!pwsh 
# Powershell kode
Write-Host "Building Project"
dotnet build Efcore.sln | Out-File -FilePath BuildOutput.txt
Write-Host "Project build completed"

#!C#
// Nuget Import 
#r "nuget: Bogus"
#r "nuget: Microsoft.EntityFrameworkCore"
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "DataLayer\bin\Debug\net7.0\DataLayer.dll"

// Usings
using Microsoft.EntityFrameworkCore;
using DataLayer;
using DataLayer.Entities;

// Dbcontext instillinger.
DbContextOptionsBuilder<BlogDbContext> DbOptions = new();
DbOptions.UseSqlite("Data Source=mydb.db;",o => o.MinBatchSize(1).MaxBatchSize(100));
//DbOptions.UseSqlServer("<ConnectionString Here>"); // Du kan bruge en normal database hvis du vil.

// Seeding af tabeller
using (var db = new BlogDbContext(DbOptions.Options))
{
    Console.WriteLine("Removing old database..");
    db.Database.EnsureDeleted();
    Console.WriteLine("Creating new database..");
    db.Database.EnsureCreated();
}

// seeding med instillinger
// Alt der bliver oprettet er tilfældige navne og data.
Console.WriteLine("Seeding Database.");
var users = DbOptions.Options.CreateRandomUsers(10); // der vil blive oprettet 10 brugere
var blogs = DbOptions.Options.CreateRandomBlogs(users); // hver bruger ejer en blog
var posts = DbOptions.Options.CreateRandomPosts(40,blogs,users); // 40 post over alle blogs og brugere
Console.WriteLine("Done.");

// Enable Sensitive Data Logging
DbOptions
    .EnableSensitiveDataLogging()
    .LogTo(a => a.Display(),Microsoft.Extensions.Logging.LogLevel.Information);

Building Project
Project build completed


Installed Packages Bogus, 34.0.2 Microsoft.EntityFrameworkCore, 7.0.3 Microsoft.EntityFrameworkCore.Sqlite, 7.0.3 Microsoft.EntityFrameworkCore.SqlServer, 7.0.3

Removing old database..
Creating new database..
Seeding Database.
Done.


-----
# 2. Hent fra datbase
## Simple queries
Henter data fra databasen hvor teksten skal indeholde "et".

Der der blibver konverteret til lowercase for at få ikke at søge case sensitiv.

> Note. Det er meningen at Posts og owner er Null.
> Vi kommer til det Senere.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Blogs
                    .Where(b => b.BlogName.ToLower().Contains("et"))
                    .ToList();
                    
    Blogs.Display();
}

Her gør vi det samme bare med at vi henter posts med et specifikt Blogid.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Posts
                    .Where(b => b.BlogId == 2)
                    .ToList();
                    
    Blogs.Display();
}

## Data Transformering med queries

Man hente data og tranformere til en anden klasse med ``Select()`` og så vil entity framework selv lave de nødvendige sql kommandoer.

> Observer at entity framework selv laver inner joins for at få data

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Posts
                    .Where(p => p.BlogId == 1)
                    .Select(p => new {
                        Title = p.title,
                        Description = p.desctription,
                        UserName = p.User.UserName,
                        BlogName = p.Blog.BlogName
                    }).ToList();
                    
    Blogs.Display();
}

## Eager loading

I den første demo var User og Blog Null.

Det er fordi Entityframework kun henter fra den første entitet.


In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var BlogPost = db.Posts
                    .Where(p => p.BlogId == 1)
                    .FirstOrDefault();
                    
    $"Title: {BlogPost?.title}".Display();
    $"Desctription: {BlogPost?.desctription}".Display();
    $"User: {BlogPost.User?.UserName}".Display();
    $"BlogName: {BlogPost.Blog?.BlogName}".Display();
}

Det kan løses ved at bruge Eager loading som vil tilføje entiteneten til query så den kommer med fra databasen ved at bruge ``Include()`` metoden.

In [2]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var BlogPost = db.Posts
                    .Include(p => p.Blog) // Inkluder Blog
                    .Include(p => p.User) // Inkluder Bruger
                    .Where(p => p.BlogId == 1)
                    .FirstOrDefault();
                    
    $"Title: {BlogPost?.title}".Display();
    $"Desctription: {BlogPost?.desctription}".Display();
    $"User: {BlogPost.User?.UserName}".Display();
    $"BlogName: {BlogPost.Blog?.BlogName}".Display();
}

warn: 02-03-2023 13:34:17.769 CoreEventId.SensitiveDataLoggingEnabledWarning[10400] (Microsoft.EntityFrameworkCore.Infrastructure) 
      Sensitive data logging is enabled. Log entries and exception messages may include sensitive application data; this mode should only be enabled during development.

info: 02-03-2023 13:34:17.958 RelationalEventId.CommandExecuted[20101] (Microsoft.EntityFrameworkCore.Database.Command) 
      Executed DbCommand (0ms) [Parameters=[], CommandType='Text', CommandTimeout='30']
      SELECT "p"."PostId", "p"."BlogId", "p"."UserId", "p"."desctription", "p"."title", "b"."BlogId", "b"."BlogName", "b"."UserId", "u"."UserId", "u"."Email", "u"."UserName"
      FROM "Posts" AS "p"
      INNER JOIN "Blogs" AS "b" ON "p"."BlogId" = "b"."BlogId"
      INNER JOIN "Users" AS "u" ON "p"."UserId" = "u"."UserId"
      WHERE "p"."BlogId" = 1
      LIMIT 1

Title: this product is standard.

Desctription: Quo quaerat dicta sunt est modi qui quaerat inventore. Reiciendis tempore expedita nulla id enim asperiores. Officiis voluptatem qui eveniet placeat libero ad.

User: Aliya.Nieklauson17

BlogName: Sunt voluptates nihil consequatur sunt voluptate assumenda et ut.

----
# 3. Oprettelse 

## Insert af en entry

Normalt i entityframework vil man indsætte det i din dbcontext og derefter kalder på kontextens ``SaveChanges()`` eller ``SaveChangesAsync()`` for at skrive til databasen.


In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    // opret bruger objekt
    var user = new User() 
    {
        UserName = "User",
        Email = "User@User.com"
    } ;

    Console.WriteLine("User to be inserted");
    user.Display();

    db.Users.Add(user); // tilføj til kontekst
    // db.Add(user); // Du kan også tilføje et objekt som ikke har et Dbset defineret

    db.SaveChanges(); // Skriv ændringer til database
}

using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Users
                    .Select(u => new {
                        u.UserName,
                        u.Email,
                    }).ToList();
                    
    Blogs.Display();
}

## Nested Insert

Du kan også oprette objekter inden i objekter som så bliver oprettet på samme tid.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    // opret bruger objekt
    var user = new User() 
    {
        UserName = "User",
        Email = "User@User.com"
    };

    Console.WriteLine("User to be inserted");
    user.Display();

    // Giver bruger objektet en ny blog.
    user.OwnerOf = new List<Blog>() {
        new Blog() {
            BlogName = "Blog1"
        },
    };

    Console.WriteLine("Blogs nested in User:");
    user.OwnerOf.Display();

    db.Users.Add(user); // tilføj til kontekst

    db.SaveChanges(); // Skriv ændringer til database
}

## Insert af flere entries.

Hvis du tilføjer flere entries vil den lave en query for hver entry og sende den en af gangen.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    // opret bruger objekter
    var user1 = new User() 
    {
        UserName = "User1",
        Email = "User1@User.com"
    };

    var user2 = new User() 
    {
        UserName = "User2",
        Email = "User1@User.com"
    };

    Console.WriteLine("Users:");
    user1.Display();
    user2.Display();

    // Indset objekter
    db.Users.Add(user1);
    db.Users.Add(user2);

    db.SaveChanges(); // Skriv ændringer til database
}

Du kan også insætte en liste med hjælp af ``AddRange()``

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    List<User> users = new() {
        new User() 
        {
            UserName = "User1",
            Email = "User1@User.com"
        },
        new User() 
        {
            UserName = "User2",
            Email = "User1@User.com"
        }
    };

    db.Users.AddRange(users);
    
    db.SaveChanges(); // Skriv ændringer til database
}

----
# 4. Opdatering af data

## Opdatering af en entry

For at opdatere skal man på normal vis vide hvad man skal opdatere.

Som standard in entity framework vil man hente og derefter rette de properties man gerne vil ændre

entitiy framework vil automatisk holde øje med objektet og opdatere den når ``SaveChanges()`` bliver kaldt.

> hvis du kører koden flere gange med den samme ville du kunne se at entity framework ikke sender nogen query fordi den har fundet ud af ændringerne er den samme.

> Prøv at lav en ændring af brugernavnet og se om ændringerne kommer igennem.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    Console.WriteLine("Get User");
    // Hent bruger fra database
    User user = db.Users.Where(u => u.UserId == 1).FirstOrDefault();

    user.Display();
    
    // retter Brugernavn til noget andet
    user.UserName = "SomeCoolUSer";

    Console.WriteLine("Save User");

    // Gæm ændringer
    db.SaveChanges();

    user.Display();
}

## Opdatering med brug af ChangeTracker

I nogle tilfælde vil man gerne have kontrol over hvike fælter der skal opdateres.

Der har dbconext der hedder ``Entry()`` som man kan bruge til at rette en en entity som enten er Added,Modified,Detached eller Deleted.

> prøv at se hvad der sker hvis du retter entity state på en af felterne.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    Console.WriteLine("Get User");
    // Hent bruger fra database
    User user = db.Users.Where(u => u.UserId == 1).FirstOrDefault();

    user.Display();
    
    // retter Brugernavn til noget andet
    user.UserName = "SomeCoolUSer";

    db.Entry(user).Property(u => u.UserName).EntityEntry.State = EntityState.Modified; // Brugernavn bliver ændret
    db.Entry(user).Property(u => u.Email).EntityEntry.State = EntityState.Modified; // Email bliver ændret selvon den ikke er ændret

    // De øverste kan man også gøre på en linje.
    //db.Entry(user).State = EntityState.Modified;

    Console.WriteLine("Save User");

    // Gæm ændringer
    db.SaveChanges();

    user.Display();
}

## Opdatering af en detached entry

Nogle gange kan man komme ud for at man har et objekt som skal opdateres men ikke er blevet hentet fra databasen.

I dette tilfælde kan man bruge ``Attach()`` for at kunne tilføje den til konteksten og derefter hvis det er en enkel markere hvad der skal opdateres og hvad der ikke skal opdateres med changetrackeren dat den markerer alle som modified.

> ``AsNoTracking()`` gør at den henter data og Changetrackeren vil ikke tracke de objekter som bliver hentet.

In [4]:
User user = new(); // Simulerer et Detached objekt

using (var db = new BlogDbContext(DbOptions.Options))
{
    Console.WriteLine("Get User");

    // Hent bruger fra database
    user = db.Users.Where(u => u.UserId == 3).AsNoTracking().FirstOrDefault();

    user.Display();
}

user.Email = "NewEmail@User.dk";

using (var db = new BlogDbContext(DbOptions.Options))
{
    // returnerer en entitycontekst som du kan bruge til at sætte entitystate
    var userEntry = db.Attach(user);

    userEntry.Property(u => u.UserId).EntityEntry.State = EntityState.Unchanged;
    userEntry.Property(u => u.UserName).EntityEntry.State = EntityState.Unchanged;
    userEntry.Property(u => u.Email).EntityEntry.State = EntityState.Modified;

    db.SaveChanges();
}

Get User


info: 02-03-2023 13:52:18.383 RelationalEventId.CommandExecuted[20101] (Microsoft.EntityFrameworkCore.Database.Command) 
      Executed DbCommand (0ms) [Parameters=[], CommandType='Text', CommandTimeout='30']
      SELECT "u"."UserId", "u"."Email", "u"."UserName"
      FROM "Users" AS "u"
      WHERE "u"."UserId" = 3
      LIMIT 1

UserId,UserName,Email,OwnerOf
3,Elias71,Elias_Knorr17@yahoo.com,<null>


info: 02-03-2023 13:52:18.397 RelationalEventId.CommandExecuted[20101] (Microsoft.EntityFrameworkCore.Database.Command) 
      Executed DbCommand (0ms) [Parameters=[@p2='3', @p0='NewEmail@User.dk' (Nullable = false) (Size = 16), @p1='Elias71' (Nullable = false) (Size = 7)], CommandType='Text', CommandTimeout='30']
      UPDATE "Users" SET "Email" = @p0, "UserName" = @p1
      WHERE "UserId" = @p2
      RETURNING 1;